# Example 1: Query a table with SQL and magic commands

In [1]:
# Install dependencies if needed:
# !pip install google-cloud-bigquery pandas

from google.cloud import bigquery
import pandas as pd
from typing import List, Optional
import numpy as np

# Initialize client (assumes GOOGLE_APPLICATION_CREDENTIALS is set)
client = bigquery.Client(project="olist-etl-pipeline")

def profile_df(df: pd.DataFrame,
               name: str = "",
               pk: Optional[List[str]] = None,
               sample_uniques: int = 20):
    """
    Quick EDA for one table.
    """
    print(f"\n{'='*80}\nTABLE: {name} — shape {df.shape}\n{'='*80}")
    display(df.head())

    # ⇢ dtypes
    print("\n--- column dtypes -----------------------------------")
    print(df.dtypes.sort_index())

    # ⇢ missing values
    print("\n--- missing values ----------------------------------")
    na = df.isna().sum().to_frame("missing_cnt")
    na["missing_pct"] = (na["missing_cnt"] / len(df) * 100).round(2)
    display(na.sort_values("missing_cnt", ascending=False))

    # ⇢ numeric summary
    num_cols = df.select_dtypes(include=np.number).columns
    if num_cols.any():
        print("\n--- numeric summary --------------------------------")
        display(df[num_cols].describe().T)

    # ⇢ unique counts (categoricals / ids)
    cat_cols = df.select_dtypes(include="object").columns
    if cat_cols.any():
        print("\n--- unique counts ----------------------------------")
        uniq = df[cat_cols].nunique().sort_values(ascending=False)
        display(uniq.to_frame("n_unique").head(20))

    # ⇢ duplicates on PK
    if pk:
        dup_cnt = df.duplicated(subset=pk).sum()
        print(f"\n--- duplicate check on {pk} ------------------------")
        print(f"Rows violating PK: {dup_cnt}")

TABLE = "olist-etl-pipeline.staging.geolocation"

query = f"SELECT * FROM `{TABLE}`"
df_geo = client.query(query).to_dataframe()
profile_df(df_geo, name="geolocation")


TABLE: geolocation — shape (1000163, 5)


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP



--- column dtypes -----------------------------------
geolocation_city                object
geolocation_lat                float64
geolocation_lng                float64
geolocation_state               object
geolocation_zip_code_prefix      Int64
dtype: object

--- missing values ----------------------------------


,missing_cnt,missing_pct
geolocation_zip_code_prefix,0,0.0
geolocation_lat,0,0.0
geolocation_lng,0,0.0
geolocation_city,0,0.0
geolocation_state,0,0.0



--- numeric summary --------------------------------


,count,mean,std,min,25%,50%,75%,max
geolocation_zip_code_prefix,1000163.0,36574.166466,30549.33571,1001.0,11075.0,26530.0,63504.0,99990.0
geolocation_lat,1000163.0,-21.176153,5.715866,-36.605374,-23.603546,-22.919377,-19.97962,45.065933
geolocation_lng,1000163.0,-46.390541,4.269748,-101.466766,-48.573172,-46.637879,-43.767709,121.105394



--- unique counts ----------------------------------


,n_unique
geolocation_city,8011
geolocation_state,27
